In [20]:
import datetime
import gpm_api
from gpm_api.visualization.plot_3d import plot_3d_radar_isosurface

# Example - Hurricane

In [21]:
# Define analysis time period
start_time = datetime.datetime.strptime("2020-10-28 08:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2020-10-28 09:00:00", "%Y-%m-%d %H:%M:%S")

# Define products to analyze
product = "2A-DPR"    # 2A-PR
product_type = "RS"  
version = 7

# Load the 2A-DPR dataset
# - If scan_mode is not specified, it automatically load one! 
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
)
ds

bbox = [-94, -89, 22.5, 27.5]
ds_patch = ds.gpm_api.crop(bbox)

'scan_mode' has not been specified. Default to FS.


Visualize the 3D Ku band reflectivity isosurface using pyvista 

In [ ]:
plot_3d_radar_isosurface(ds_patch, 
                         radar_frequency="Ku",
                         isovalues=[30,40,50],
                         opacities=[0.3, 0.5, 1],
                         )

# Example - Another random event

In [22]:
# Specify the time period you are interested in 
start_time = datetime.datetime.strptime("2020-07-05 02:00:00", "%Y-%m-%d %H:%M:%S")
end_time = datetime.datetime.strptime("2020-07-05 06:00:00", "%Y-%m-%d %H:%M:%S")
# Specify the product and product type 
product = "2A-DPR"    # 2A-PR
product_type = "RS"   
# Specify the version
version = 7

# Load the 2A-DPR dataset
# - If scan_mode is not specified, it automatically load one! 
ds = gpm_api.open_dataset(
    product=product,
    product_type=product_type,
    version=version,
    start_time=start_time,
    end_time=end_time,
)
ds

isel_dict = {"cross_track": slice(0,49), "along_track": slice(884, 984)}
ds_patch = ds.isel(isel_dict)

'scan_mode' has not been specified. Default to FS.


Visualize the 3D Ku band reflectivity isosurface using pyvista

In [ ]:
plot_3d_radar_isosurface(ds_patch, 
                         radar_frequency="Ku",
                         isovalues=[20,30,40],
                         opacities=[0.05, 0.4, 1],
                         add_colorbar=True,
                         )